This notebooks seeks to replicate the results in ["Exact solution and Majorana zero mode generation on a Kitaev chain composed out of noisy qubits"](https://arxiv.org/abs/2108.07235), and will attempt to use native Qiskit modules in functionality as well as achieve some "best practices" when running on actual IBM backends.

## Prerequisites

In [1]:
run_experiment = False
dynamical_decoupling = True
meas_error_mitigation = False
pulse_scaling = True

### Load Qiskit and Required Libraries

In [2]:
from qiskit import IBMQ, transpile, assemble, schedule
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import XGate, YGate
from qiskit.quantum_info import Operator, Statevector

from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

from qiskit.opflow import (I, X, Y, Z, Zero, One, MatrixEvolution, PauliTrotterEvolution, Suzuki,
                           StateFn, Zero, One, PauliExpectation, PauliOp, SummedOp, OperatorBase)

from qiskit.transpiler import PassManager, InstructionDurations
from qiskit.transpiler.passes import TemplateOptimization, ALAPSchedule, DynamicalDecoupling
from qiskit.transpiler.passes.calibration import RZXCalibrationBuilder, rzx_templates

from qiskit.converters import circuit_to_dag, dag_to_circuit # for bespoke transpilation
from qiskit.dagcircuit import DAGCircuit, DAGNode

from qiskit.scheduler.config import ScheduleConfig
from qiskit.visualization import plot_circuit_layout, plot_error_map, timeline_drawer

import copy
import numpy as np
import scipy.linalg as lng
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [5, 5]

### Load IBM Quantum Account

In [3]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibm_lagos')

### Load Backend Information (for Pulse)

In [4]:
backend_config = backend.configuration()
dt = backend_config.dt
meas_map = backend_config.meas_map

backend_defaults = backend.defaults()
inst_sched_map = backend_defaults.instruction_schedule_map

sched_config = ScheduleConfig(inst_sched_map, meas_map, dt)

# Build Circuits from Model Hamiltonian

## Define the System Hamiltonian

The full $n$-site Kitaev Hamiltonian is given by
$$
H = \sum_{k=1, n} \mu_k c_k^\dagger c_k - \sum_{\langle kj \rangle} \left(
t_{kj} c_k^\dagger c_j + t_{jk} c_j^\dagger c_k - \Delta_{kj} c_k^\dagger c_j^\dagger - \Delta_{kj} c_j c_k\right),
$$
and for simplicity no local variations will be assumed, i.e. $\mu_k \equiv \mu, t_{kj} \equiv t, \Delta_{kj} \equiv \Delta$. A number of expectation values will be calculated/simulated from the "exactly [prepared] eigenstates of quadratic Hamiltonians" (see below), namely energy
$$ E = \langle \psi | H | \psi \rangle, $$
Majorana edge correlation function
$$ \langle \psi | i\gamma_1\gamma_{2n} | \psi \rangle $$
(dunno where the $2n$ comes from here),
Majorana site correlation function
$$ \langle \psi | i\gamma_1\gamma_k | \psi \rangle, $$
Majorana parity operator
$$ \langle \mathcal{P} \rangle = \langle \psi | \prod_{k=1}^n \left(1 - 2c_k^\dagger c_k \right) | \psi\rangle, $$
and particle number operator
$$ \langle N \rangle = \langle \psi | \sum_{k=1}^n c_k^\dagger c_k | \psi \rangle.$$

It seems that these $|\psi\rangle$ states are created as these fermionic Guassian states (which are "exactly-prepared states of the quadratic Hamiltonian"). In this case let's define an operator $U_{\rm fGs}$ that prepares these states from the initial state, i.e.
$$
|\psi \rangle = U_{\rm fGs} |000\rangle.
$$$
Anyway, let's write it all in Qiskit Nature.

In [5]:
hm = sum(FermionicOp(label) for label in ['IIN', 'INI', 'NII']) # also the number operator
ht = -sum(FermionicOp(label) for label in ['I+-', 'I-+', '+I-', '-I+', '+-I', '-+I'])
hD = sum(FermionicOp(label) for label in ['I++', 'I--', '+I+', '-I-', '++I', '--I'])

# Majorana operators, either gamma = c^\dagger + c or 
# c = i(c^\dagger - c), should be opposite on opposite ends of Kitaev chain (?)
gamma0 = sum(FermionicOp(label) for label in ['II+', 'II-'])
gamma1 = sum(FermionicOp(label) for label in ['I+I', 'I-I'])
gamma2 = sum(FermionicOp(label) for label in ['+II', '-II'])

# and/or
#gamma0 = 1j*(FermionicOp('II+') - FermionicOp('II-'))
#gamma1 = 1j*(FermionicOp('I+I') - FermionicOp('I-I'))
#gamma2 = 1j*(FermionicOp('+II') - FermionicOp('-II'))

### Transform Fermionic to Pauli Hamiltonian
Bravyi-Kitaev and BKSuperFast are also built into Qiskit.

In [6]:
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper) # should not give 2-qubit reduction error

hm_pauli = converter.convert(hm)
ht_pauli = converter.convert(ht)
hD_pauli = converter.convert(hD)

g0_pauli = converter.convert(gamma0)
g1_pauli = converter.convert(gamma1)
g2_pauli = converter.convert(gamma2)

In [7]:
# parameters defined here due to incompatibility with Qiskit Nature
mu = Parameter('$\mu$')
TT = Parameter('$T$')
DD = Parameter('$\Delta$')

In [8]:
ham_pauli = (mu*hm_pauli) + (TT*ht_pauli)  + (DD*hD_pauli)
#print(ham_pauli)

In [9]:
tt = Parameter('$t$')
U_ham = (tt*ham_pauli).exp_i()
#print(U_ham)

The above shows how you would get a time-evolved operator using Qiskit Nature/Opflow. But actually we just need to prepare the initial states and measure expecation values.

## Create Fermionic Gaussian States

Kevin will show us how!

## Trotterize Unitary ~Evolution~ Preparation Operator

Depending on whether we're doing this for an `opflow` simulator or running on a real circuit, we may want to Trotterize just the $U_{\rm fGs}$ or perhaps the entire expectation value. Nick is not completely clear why this matters. Here's an example of how you would do a single second-order Trotter step and get a `CircuitOp`/`QuantumCircuit` out. Note the `PauliTrotterEvolution` method thinks the Hamiltonian is not composed entirely of `Pauli`s, even though it is. This has to do with a parsing issue that Kaelyn is working to fix.

In [10]:
trot_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(U_ham)
#trot_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(U_fGs)
trot_circ = trot_op.to_circuit()
#trot_circ.draw(output='mpl', reverse_bits=True)

Evolved Hamiltonian is not composed of only Paulis, converting to Pauli representation, which can be expensive.


# Operator/Circuit Simulations

Express the initial state preparation in terms of operators.

In [12]:
gnd_state = Zero^3 # |000>

# operators for expectation values
H_obsv = ham_pauli
edge_obsv = g0_pauli*g2_pauli # ???
site_obsv = g0_pauli*g1_pauli
N_obsv = hm_pauli
P_obsv = I^3 - 2*N_obsv
obsv = H_obsv # for example

# prepare state and calculate expectation operator
prep_and_obsv = (U_fGs @ gnd_state).adjoint() @ obsv @ U_fGs @ gnd_state

AttributeError: 'int' object has no attribute 'num_qubits'

In [ ]:
# set model parameters
param_bind = {
    TT: -0.1,
    DD: 1
}

# mu range
mu_range = np.linspace(0, 3.1, 32)

## Opflow Exact Preparation

Sometimes you can bind all the `mu`'s at once, sometimes you cannot. Nick does not know why.

In [ ]:
diag_meas_op = PauliExpectation().convert(prep_and_obsv.bind_parameters(param_bind))

exp_exact = []
for mu_set in mu_range:
    simple_exps = diag_meas_op.bind_parameters({mu: mu_set})
    exp_exact.append(simple_ham_exps.eval())

## Opflow Trotter Preparation

In [ ]:
trot_steps = 5
sim_trot_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=trot_steps)).convert(prep_and_obsv)

In [ ]:
diag_meas_op = PauliExpectation().convert(sim_trot_op.bind_parameters(param_bind))
simple_ham_exps = diag_meas_op.bind_parameters({mu: list(mu_range)})
exp_trots = simple_ham_exps.eval() # does this cast to something plottable easily?

## Statevector Preparation

In [ ]:
psi = qi.Statevector.from_instruction(trot_circ).data
exp_op = obsv.bind_parameters(param_bind).to_matrix().data

exp_sv = []
for mu in mu_range:
    exp_sv.append(np.dot(np.conjugate(psi), np.dot(exp_op, psi)))

## Calculate Expectations Exactly

Use something fancy like QuTip or Dynamics?

# Transpile Circuits to Quantum Backend

## *Incredibly* useful notes on what we're doing

Transpilation will take place "by hand" so that we can introduce the template optimization at the correct point. Each *pass* of the transpiler is classified as either an analysis or transformation pass. Template optimization consists of two passes:
- `TemplateOptimization` is an analysis pass that adds the templates (similar to circuit equivalences), in this case specified by `rzx_templates()` 
- `RZXCalibrationBuilder` is a transformation pass that replaces $ZX(\theta)$ gates with the locally-equivalent scaled Pulse gates

The **order** of transpilation and where the backend information such as layout and native gate set are incredibly important and the following heuristics were able to get this to work:

- The circuit must be transpiled to an `initial_layout` since the controlled-`RZGate` operations go across unconnected qubit pairs. At this point it seems best to leave the `basis_gate` set the same as that used in Trotterization.

- Next the `TemplateOptimization` can be run (since the simplication will respect qubit layout), running on Nick's dev fork branch `template-param-expression` (Qiskit Terra [PR 6899](https://github.com/Qiskit/qiskit-terra/pull/6899)) will allow `Parameter`s to be passed through this step.

- The `TemplateOptimization` will miss some patterns because the template parameters will conflict with finding a maximal match (Qiskit Terra [Issue 6974](https://github.com/Qiskit/qiskit-terra/issues/6974)). Here we run **Bespoke Passes** that combine consecutive gates with `Parameter`s (`RZGate`s in this case) and force $ZZ$-like patterns to match and be replated with the inverse from the template.

- Heavily transpile (`optimization_level=3`) the circuit without reference to basis gates (this was necessary for some reason?)

- Final bespoke combination of `RZGate`s.

- If there are still patterns of CNOT-singles-CNOT that could be optimized, Nick has code to do this by force.

## Backend Information

In [ ]:
plot_error_map(backend)

In [ ]:
qr = QuantumRegister(backend_config.num_qubits, 'q')
cr = ClassicalRegister(backend_config.num_qubits, 'c')
initial_layout = [3, 1, 2] # favorite three qubits
native_gates = ['rz', 'sx', 'rzx', 'x']

In [ ]:
avg_gate_error = 0
for ii in range(len(initial_layout)-1):
    q0 = initial_layout[ii]
    q1 = initial_layout[ii+1]
    avg_gate_error += backend.properties().gate_property('cx')[(q0, q1)]['gate_error'][0]

avg_gate_error /= len(initial_layout)-1
print('Avg 2-qubit gate error is '+str(avg_gate_error))

## Template Optimization and Basic Transpilation

In [ ]:
trot_circ1 = transpile(trot_circ, optimization_level=0)

if pulse_scaling:
    pass_ = TemplateOptimization(**rzx_templates.rzx_templates()) 
    trot_circ2 = PassManager(pass_).run(trot_circ1)
    trot_circ3 = transpile(trot_circ2, basis_gates=native_gates,
                      backend=backend, initial_layout=initial_layout)
else:
    trot_circ3 = transpile(trot_circ1, basis_gates=['sx', 'rz', 'cx'],
                      backend=backend, initial_layout=initial_layout)

#trot_circ3.draw(output='mpl', idle_wires=False)

## Bespoke Transpilation Time

So far, just doing one to combine consecutive gates. Does not look like modulo $2\pi$ or forcet `CNOT-single-CNOT` is necessary here.

### Combine Consectutive Gates Pass

In [ ]:
def combine_runs(dag: DAGNode, gate_str: str) -> DAGCircuit:
    runs = dag.collect_runs([gate_str])
    for run in runs:
        partition = []
        chunk = []
        for ii in range(len(run)-1):
            chunk.append(run[ii])

            qargs0 = run[ii].qargs
            qargs1 = run[ii+1].qargs

            if qargs0 != qargs1:
                partition.append(chunk)
                chunk = []

        chunk.append(run[-1])
        partition.append(chunk)

        # simplify each chunk in the partition
        for chunk in partition:
            theta = 0
            for ii in range(len(chunk)):
                theta += chunk[ii].op.params[0]

            # set the first chunk to sum of params
            chunk[0].op.params[0] = theta

            # remove remaining chunks if any
            if len(chunk) > 1:
                for nn in chunk[1:]:
                    dag.remove_op_node(nn)
    return dag

### Run Bespoke Passes

In [ ]:
dag = circuit_to_dag(trot_circ3)
dag = combine_runs(dag, 'rz')
dag = combine_runs(dag, 'rzx')
trot_circ4 = dag_to_circuit(dag)
trot_circ4.draw(output='mpl', idle_wires=False)

## Game Plan
The above circuit is as transpiled as possible without binding parameters and adding the calibrations for the `RZXGate`s. This will form the unit of the sweeps we run.

# Build Experiment

In [13]:
trot_unit = trot_circ4

exp_str = 'my_exp_string' # like 'mu_sweep'

NameError: name 'trot_circ4' is not defined

## Set Model Hamiltonian Parameters

### Bind Parameters and Append Circuits

In [ ]:
if exp_str == 'mu_sweep':
    mu_circs = []
    
    # bind remaining parameters
    for mu_set in mu_range:
        param_bind[mu] = mu_set
        mu_circs.append(trot_circ4.bind_parameters(param_bind))
        # compose circuits as necessary

## Final Transpilation Steps

Nick thinks this should probably be simplified, it takes too much time and we probably just need basic gate simplification parts.

In [ ]:
if pulse_scaling:
    mu_circs_sca_t = transpile(mu_circs, backend, basis_gates=native_gates)
    
mu_circs_dig_t = transpile(mu_circs, backend)

In [ ]:
if pulse_scaling:
    pass_ = RZXCalibrationBuilder(backend)
    mu_circs_sca_t1 = PassManager(pass_).run(mu_circs_sca_t)

## Dynamical Decoupling

Following [this tutorial](https://qiskit.org/documentation/stubs/qiskit.transpiler.passes.DynamicalDecoupling.html), we can automatically add dynamical decoupling sequences. Currently this seems a bit limited to "native" Qiskit gates, and the duration information must be pulled from the `inst_sched_map`. Nick has raised this as Qiskit Terra [Issue 7400](https://github.com/Qiskit/qiskit-terra/issues/7400).

## Retrieve Gate Durations

In [ ]:
if dynamical_decoupling:
    inst_durs = []

    # single qubit gates
    for qubit in range(backend_config.num_qubits):
        for inst_str in inst_sched_map.qubit_instructions(qubits=[qubit]):
            inst = inst_sched_map.get(inst_str, qubits=[qubit])
            inst_durs.append((inst_str, qubit, inst.duration))
            if inst_str == 'x':
                inst_durs.append(('y', qubit, inst.duration))

    # two qubit gates
    for qc in range(backend_config.num_qubits):
        for qt in range(backend_config.num_qubits):
            for inst_str in inst_sched_map.qubit_instructions(qubits=[qc, qt]):
                inst = inst_sched_map.get(inst_str, qubits=[qc, qt])
                inst_durs.append((inst_str, [qc, qt], inst.duration))

    durations = InstructionDurations(inst_durs)

In [ ]:
#res_circ_nodd = transpile(res_circ_scaled_trans1, backend, scheduling_method='alap')

In [ ]:
if dynamical_decoupling:
    # balanced X-X sequence on all qubits
    dd_X2_sequence = [XGate(), XGate()]
    pm = PassManager([ALAPSchedule(durations),
                      DynamicalDecoupling(durations, dd_X2_sequence)])
    mu_circs_ddx2 = pm.run(mu_circs_sca_t1)

Petar actually does an `Xp`-`Xm` sequence that is described at the end of Section IV of the [QV 64 paper](http://arxiv.org/abs/2008.08571), which is nominally the same but actually different as far as pulses are concerned.

In [ ]:
if dynamical_decoupling:
    dd_XY4_sequence = [XGate(), YGate(), XGate(), YGate()]
    pm = PassManager([ALAPSchedule(durations),
                      DynamicalDecoupling(durations, dd_XY4_sequence)])
    mu_circs_ddxy4 = pm.run(mu_circs_sca_t1)

In [ ]:
if dynamical_decoupling:
    # for some reason dd sequence does not seem to be inserted until after entangling gate,
    # although end looks fine (should probably raise Terra issue)
    start_time = 0 #880000
    time_window = 30000
    time_range=[start_time, start_time+time_window]
    timeline_drawer(mu_circs_ddxy4[-1], time_range=time_range)

### Add Calibrations for Missing Gates

The`YGate` is not a basis gate of our backend (unlike 'x'). We can build a pulse schedule from 'x' and add it to the circuits.

In [ ]:
if dynamical_decoupling:
    for qubit in range(backend_config.num_qubits):
        with pulse.build('y gate for qubit '+str(qubit)) as sched:
            with pulse.phase_offset(np.pi/2, DriveChannel(qubit)):
                x_gate = inst_sched_map.get('x', qubits=[qubit])
                pulse.call(x_gate)

            for circ in mu_circs_ddxy4:
                circ.add_calibration('y', [qubit], sched)

In [ ]:
# okay this does not work for some reason, but confirmed it on a simpler circuit
#schedule(mu_circs_ddxy4[-1], backend).draw(time_range=time_range)

## Compare digital and scaled circuits

In [ ]:
if pulse_scaling:
    circ_num = -1
    scaled_sched = schedule(mu_circs_sca_t1[circ_num], backend)
    basis_sched = schedule(mu_circs_dig_t[circ_num], backend)

### Count Operations

In [ ]:
if pulse_scaling:
    mu_circs_sca_t1[circ_num].count_ops()

In [ ]:
mu_circs_dig_t[circ_num].count_ops()

In [ ]:
if pulse_scaling:
    dag = circuit_to_dag(mu_circs_sca_t1[circ_num])
    rzx_runs = dag.collect_runs(['rzx'])

    est_fid_rzx = 1
    for rzx_run in rzx_runs:
        angle = rzx_run[0].op.params[0]
        this_rzx_error = (abs(float(angle))/(np.pi/2))*avg_gate_error
        est_fid_rzx *= (1-this_rzx_error)

    print('Scaled Circuit estimated fidelity is %2.f%%' % (est_fid_rzx*100))

In [ ]:
num_cx = mu_circs_dig_t[circ_num].count_ops()['cx']
est_fid_dig = (1-avg_gate_error)**num_cx
print('Digital Circuit estimated fidelity is %2.f%%' % (est_fid_dig*100))

### Look at Resulting Schedules

In [ ]:
if pulse_scaling:
    print('Scaled schedule takes '+str(scaled_sched.duration)+'dt')
    
print('Digital schedule takes '+str(basis_sched.duration)+'dt')

In [ ]:
time_range=[0,4000]
if pulse_scaling:
    scaled_sched.draw(time_range=time_range)

In [ ]:
basis_sched.draw(time_range=time_range)

Question: How is the duration being calculated when the parameters for trans_circ_dig have not been set? 

NTB: I don't know, just noticed this as well in another notebook. My initial guess is that 1q-gates have 2 microwave pulses and the RZX are calculated to have 2 CNOTs, but this needs to be confirmed.

# Run on Quantum Hardware

In [ ]:
from qiskit.tools.monitor import job_monitor

if run_experiment:
    # run the job on a real backend
    if dynamical_decoupling:
        job = backend.run(mu_circs_sca_t1 + mu_circs_ddx2 + mu_circs_ddxy4, 
                          job_name='mzm generation', meas_level=2, shots=10000) 
    else:
        job = backend.run(mu_circs_sca_t1, job_name='mzm generation', meas_level=2, shots=10000) 
    
    print(job.job_id())
    job_monitor(job)

## Or Retrieve from Previous Run

In [ ]:
if not run_experiment:
    job1 = backend.retrieve_job('') # mu_sweep
    job = job1

In [ ]:
Result = job.result().get_counts()

# Analyze Results

## Retrieve Results from Actual Job

## Plot Results and Sims

This nominally should work without having the experimental data as well (i.e., for checks).

## Save Results

# Qiskit Version Table

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table